<a href="https://colab.research.google.com/github/JayaManasa/multimodal-finetuning/blob/main/LLaVA_image_text_prompt_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade -q accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-4dh39i0n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-4dh39i0n
  Resolved https://github.com/huggingface/transformers.git to commit 293e6271c69a48b6a66e68978dd3d37601c04c63
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9972378 sha256=fa4c96fe4cec44b384e6eb63953b3280f9d29a64d058968b7e353e80cd634757
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3n44xce/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Su

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.44.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [3]:
import requests
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
import torch


In [4]:
# Set up quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load model and processor
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [5]:
from PIL import Image
import os

def process_image(image_path, description, prompt):
    # Open the local image file
    image = Image.open(image_path)

    # Prepare input
    text_input = f"USER: <image>\n{description}\n{prompt}\nASSISTANT:"
    inputs = processor(text=text_input, images=image, return_tensors="pt").to("cuda")

    # Generate output
    output = model.generate(**inputs, max_new_tokens=200)

    # Decode and return the generated text
    return processor.batch_decode(output, skip_special_tokens=True)[0]

# Update the image_data list to use "path" instead of "url"
image_data = [
    {
        "path": "/content/sample_images/pexels-serinus-27879531.jpg",
        "description": "A dramatic, fog-covered mountain landscape with steep cliffs and rolling green hills. The sky is overcast with dense, low-hanging clouds, creating a misty and moody atmosphere. The terrain is grassy, with a rocky cliffside dropping sharply into a valley below, giving a sense of depth and scale. The distant mountains are partially obscured by the fog, adding to the sense of mystery and isolation in this rugged, untouched natural environment.",
        "prompt": "Please describe this image in detail, along with how many people present"
    },
    {
        "path": "/content/sample_images/pexels-shane-ramos-87262929-8984849.jpg",
        "description": "A tropical waterfall surrounded by dense, lush greenery. The waterfall cascades down a cliff, framed by large leaves and tropical plants. A faint rainbow is visible in the mist near the top of the waterfall, adding a magical and serene touch to the vibrant natural scene. The vegetation is thick, creating a jungle-like setting, and the waterfall appears powerful yet peaceful as it tumbles into the green surroundings",
        "prompt": "Please describe this image in detail, along with how many people present."
    },
        {
        "path": "/content/sample_images/IMG_6516.HEIC",
        "description": "People posing as a group",
        "prompt": "Please describe this image in detail, describe the action being performed in the image"
    }
]

def main():
    # Process each image
    for idx, data in enumerate(image_data, 1):
        if os.path.exists(data["path"]):
            result = process_image(data["path"], data["description"], data["prompt"])
            print(f"\nResult for Image {idx}:")
            print(result)
        else:
            print(f"\nImage {idx} not found at path: {data['path']}")

if __name__ == "__main__":
    main()

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.



Result for Image 1:
USER:  
A dramatic, fog-covered mountain landscape with steep cliffs and rolling green hills. The sky is overcast with dense, low-hanging clouds, creating a misty and moody atmosphere. The terrain is grassy, with a rocky cliffside dropping sharply into a valley below, giving a sense of depth and scale. The distant mountains are partially obscured by the fog, adding to the sense of mystery and isolation in this rugged, untouched natural environment.
Please describe this image in detail, along with how many people present
ASSISTANT: The image depicts a breathtaking mountain landscape with steep cliffs and rolling green hills. The sky is overcast with dense, low-hanging clouds, creating a misty and moody atmosphere. The terrain is grassy, with a rocky cliffside dropping sharply into a valley below, giving a sense of depth and scale. The distant mountains are partially obscured by the fog, adding to the sense of mystery and isolation in this rugged, untouched natural e

UnidentifiedImageError: cannot identify image file '/content/sample_images/IMG_6516.HEIC'

https://colab.research.google.com/drive/17D_UacvJgf1bylxRauzzjADJVszh8UYh#scrollTo=8ONvB7YctVMm&line=1&uniqifier=1